# 샌드위치 맛집 분석
* 웹스크래핑(Web Scraping)
* 스크레이핑이란 웹 사이트에서 데이터를 추출하고, 원하는 정보를 추출하는 것입니다.

- BeautifulSoup
    - .prettify() : html 페이지의 내용을 전체 다 봄
    - .find_all('p') : 접근해야 할 태그를 알고 잇다면 find, find_all 사용

## 1. 라이브러리 호출

In [103]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

import re # 정규식

import requests
from requests.compat import urljoin # url이 길 경우, 잘라서 합치려고

from tqdm import tqdm_notebook # tqdm_notebook: 반복문이 잘 실행되는지 확인
import time # timesleep으로 tqdm_notebook잘나타내주기위해

import folium # 시각화
import googlemaps # 구글맵API이용

import numpy as np # 계산

## 2. 데이터로드

In [113]:
url_base = 'https://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = Request(url_base + url_sub, headers={'User-Agent':'Mozilla/5.0'})
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")
soup

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<title>The 50 Best Sandwiches in Chicago – Chicago Magazine</title>
<style type="text/css">
					.heateor_sss_button_instagram span.heateor_sss_svg,a.heateor_sss_instagram span.heateor_sss_svg{background:radial-gradient(circle at 30% 107%,#fdf497 0,#fdf497 5%,#fd5949 45%,#d6249f 60%,#285aeb 90%)}
						div.heateor_sss_horizontal_sharing a.heateor_sss_button_instagram span{background:#000!important;}div.heateor_sss_standard_follow_icons_container a.heateor_sss_button_instagram span{background:#000;}
										.heateor_sss_horizontal_sharing .heateor_sss_svg,.heateor_sss_standard_follow_icons_container .heateor_sss_svg{
							background-color: #000!important;
				background: #000!important;
							color: #fff;
						border-width: 0px;
			border-style: solid;
			border-color: transparent;
		}
					.heateor_sss_horiz

## 3. 분석시작

In [114]:
soup.find_all('div','sammy')

[<div class="sammy" style="position: relative;">
 <div class="sammyRank">1</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
 Old Oak Tap<br/>
 <em>Read more</em> </a></div>
 </div>, <div class="sammy" style="position: relative;">
 <div class="sammyRank">2</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/"><b>Fried Bologna</b><br/>
 Au Cheval<br/>
 <em>Read more</em> </a></div>
 </div>, <div class="sammy" style="position: relative;">
 <div class="sammyRank">3</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/"><b>Woodland Mushroom</b><br/>
 Xoco<br/>
 <em>Read more</em> </a></div>
 </div>, <div class="sammy" style="position: relative;">
 <div class="sammyRank">4</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-C

In [119]:
tmp_one = soup.find_all('div','sammy')[0]
tmp_one

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
</div>

In [121]:
print(tmp_one.find(class_='sammyRank'))
tmp_one.find(class_='sammyRank').get_text()

<div class="sammyRank">1</div>


'1'

In [131]:
print(tmp_one.find(class_='sammyListing'))
print("*"*40)
print("*"*40)
# 메뉴와 가게이름
tmp_one.find(class_='sammyListing').get_text() 

<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
****************************************
****************************************


'BLT\nOld Oak Tap\nRead more '

In [144]:
print(tmp_one.find('a'))
print("*"*40)
print("*"*40)
tmp_one.find('a')['href']

<a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a>
****************************************
****************************************


'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

### 3-1. 정규식활용하여 데이터 추출

In [149]:
# 정규식
import re
tmp_string = tmp_one.find(class_="sammyListing").get_text()
tmp_string
print(re.split(('\n|\r\n'), tmp_string))
print(re.split(('\n|\r\n'), tmp_string)[0]) # 메뉴 이름
print(re.split(('\n|\r\n'), tmp_string)[1]) # 가게 이름

['BLT', 'Old Oak Tap', 'Read more ']
BLT
Old Oak Tap


### 3-2. 절대 경로로 잡힌 url은 그대로 두고 상대경로로 잡힌 url은 절대경로로 변경

In [172]:
import requests
from requests.compat import urljoin

rank = [] # 순위
main_menu = [] # 메인 메뉴 이름
cafe_name = [] # 카페 이름
url_add = [] # url저장

list_soup = soup.find_all('div','sammy')

i = 0
for item in list_soup:
    rank.append(item.find_all(class_="sammyRank")[i].get_text())
    tmp_string = item.find_all(class_="sammyListing")[i].get_text()
    
    main_menu.append(re.split(('\n|\r\n'),tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'),tmp_string)[1])
    url_add.append(urljoin(url_base,item.find('a')['href'])) # 절대경로로 만들기
#     url_add.append(item.find('a')['href'])

#test
for i in range(len(rank)):
    print(f'{main_menu[i]} in {cafe_name[i]} -> {url_add[i]}')
    print()

BLT in Old Oak Tap -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/

Fried Bologna in Au Cheval -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/

Woodland Mushroom in Xoco -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/

Roast Beef in Al’s Deli -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/

PB&L in Publican Quality Meats -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/

Belgian Chicken Curry Salad in Hendrickx Belgian Bread Crafter -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Hendrickx-Belgian-Bread-Crafter-Belgian-Chicken-Curry-Salad/

Lobster Roll in Acadia -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches

In [173]:
from tqdm import tqdm_notebook # tqdm_notebook: 반복문이 잘 실행되는지 확인
import time

rank = [] # 순위
main_menu = [] # 메인 메뉴 이름
cafe_name = [] # 카페 이름
url_add = []

list_soup = soup.find_all('div','sammy')
bar_total = tqdm_notebook(list_soup)

i = 0
for item in bar_total:
    rank.append(item.find_all(class_="sammyRank")[i].get_text())
    tmp_string = item.find_all(class_="sammyListing")[i].get_text()
    
    main_menu.append(re.split(('\n|\r\n'),tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'),tmp_string)[1])
    url_add.append(urljoin(url_base,item.find('a')['href']))
    
    time.sleep(0.05)

for i in range(len(rank)):
    print(f'{main_menu[i]} in {cafe_name[i]} -> {url_add[i]}')



BLT in Old Oak Tap -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/

Fried Bologna in Au Cheval -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/

Woodland Mushroom in Xoco -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/

Roast Beef in Al’s Deli -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/

PB&L in Publican Quality Meats -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/

Belgian Chicken Curry Salad in Hendrickx Belgian Bread Crafter -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Hendrickx-Belgian-Bread-Crafter-Belgian-Chicken-Curry-Salad/

Lobster Roll in Acadia -> https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiche

### 3-3. dataframe으로 변환

In [177]:
import pandas as pd
data={
    'Rank'  : rank
    ,'Menu' : main_menu
    ,'Cafe' : cafe_name
    ,'URL'  : url_add
}
df = pd.DataFrame(data, columns=['Rank','Cafe','Menu','URL'])
df.head(3)

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...


### 3-4. to_csv로 저장 테스트

In [179]:
df.to_csv('../data/03. best_sandwiches_chicago.csv', sep=',', encoding='utf-8')

### 3-5. to_csv로 저장 테스트

In [185]:
df = pd.read_csv('../data/03. best_sandwiches_chicago.csv', index_col=0)
df

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...
5,6,Hendrickx Belgian Bread Crafter,Belgian Chicken Curry Salad,https://www.chicagomag.com/Chicago-Magazine/No...
6,7,Acadia,Lobster Roll,https://www.chicagomag.com/Chicago-Magazine/No...
7,8,Birchwood Kitchen,Smoked Salmon Salad,https://www.chicagomag.com/Chicago-Magazine/No...
8,9,Cemitas Puebla,Atomica Cemitas,https://www.chicagomag.com/Chicago-Magazine/No...
9,10,Nana,Grilled Laughing Bird Shrimp and Fried Po’ Boy,https://www.chicagomag.com/Chicago-Magazine/No...


### 3-6. URL에서 특정 정보 찾아오기

In [203]:
url = Request(df['URL'][0], headers={'User-Agent': 'Mozilla/5.0'})
html = urlopen(url)
soup_tmp = BeautifulSoup(html,'html.parser')
soup_tmp

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<title>1. Old Oak Tap BLT – Chicago Magazine</title>
<style type="text/css">
					.heateor_sss_button_instagram span.heateor_sss_svg,a.heateor_sss_instagram span.heateor_sss_svg{background:radial-gradient(circle at 30% 107%,#fdf497 0,#fdf497 5%,#fd5949 45%,#d6249f 60%,#285aeb 90%)}
						div.heateor_sss_horizontal_sharing a.heateor_sss_button_instagram span{background:#000!important;}div.heateor_sss_standard_follow_icons_container a.heateor_sss_button_instagram span{background:#000;}
										.heateor_sss_horizontal_sharing .heateor_sss_svg,.heateor_sss_standard_follow_icons_container .heateor_sss_svg{
							background-color: #000!important;
				background: #000!important;
							color: #fff;
						border-width: 0px;
			border-style: solid;
			border-color: transparent;
		}
					.heateor_sss_horizontal_sharing .

In [208]:
# 가격, 장소, 전화번호, 사이트
price_tmp = soup_tmp.find('p','addy').get_text()
price_tmp

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [211]:
price_tmp.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

In [216]:
print(price_tmp.split()[1:-2])
print("*"*40)
print("*"*40)
print(' '.join(price_tmp.split()[1:-2]))

['2109', 'W.', 'Chicago', 'Ave.,']
****************************************
****************************************
2109 W. Chicago Ave.,


In [217]:
df

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...
5,6,Hendrickx Belgian Bread Crafter,Belgian Chicken Curry Salad,https://www.chicagomag.com/Chicago-Magazine/No...
6,7,Acadia,Lobster Roll,https://www.chicagomag.com/Chicago-Magazine/No...
7,8,Birchwood Kitchen,Smoked Salmon Salad,https://www.chicagomag.com/Chicago-Magazine/No...
8,9,Cemitas Puebla,Atomica Cemitas,https://www.chicagomag.com/Chicago-Magazine/No...
9,10,Nana,Grilled Laughing Bird Shrimp and Fried Po’ Boy,https://www.chicagomag.com/Chicago-Magazine/No...


In [222]:
price   = []
address = []
for n in df.index[:3]:
    url = Request(df['URL'][n], headers={'User-Agent':'Mozilla/5.0'})
    html = urlopen(url)
    soup_tmp = BeautifulSoup(html, 'lxml')
    
    gettings = soup_tmp.find('p','addy').get_text()
#     print(gettings)
    price.append(gettings.split()[0][:-1]) # '.'뺴려고 -1까지
    address.append(' '.join(gettings.split()[1:-2]))

for i in range(3):
    print(f'{price[i]} in {address[i]}')


$10 in 2109 W. Chicago Ave.,
$9 in 800 W. Randolph St.,
$9.50 in 445 N. Clark St.,


In [318]:
from tqdm import tqdm_notebook
price   = []
address = []

for n in tqdm_notebook(df.index):
    url = Request(df['URL'][n], headers={'User-Agent':'Mozilla/5.0'})
    html = urlopen(url)
    soup_tmp = BeautifulSoup(html, 'lxml') # lxml이 html.parser보다 빨라서 
    
    gettings = soup_tmp.find('p','addy').get_text()

    price.append(gettings.split()[0][:-1]) # '.'뺴려고 -1까지
    address.append(' '.join(gettings.split()[1:-2]))
    time.sleep(0.03)



KeyError: 'URL'

In [224]:
len(price), len(address), len(df)

(50, 50, 50)

### 3-7. 웹에서 긁어온 데이터를 dataframe에 추가

In [229]:
df.head(3)

,Rank,Cafe,Menu,URL,Price,Address
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"


In [226]:
df['Price'] = price
df['Address'] = address
df.head(3)

In [231]:
df = df.loc[:, ['Rank','Cafe','Menu','Price','Address']] # 컬럼 순서 변경
df.set_index('Rank',inplace=True)
df.head(3)

,Cafe,Menu,Price,Address
Rank,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,"
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,"
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,"


In [234]:
df.to_csv('../data/03. best_sandwiches_chicago22.csv', sep=',', encoding='utf-8')

## 4. 그래프그리기 -Folium


In [233]:
import folium
import pandas as pd
import googlemaps
import numpy as np

In [259]:
df = pd.read_csv('../data/03. best_sandwiches_chicago22.csv',index_col=0)
df.head()

,Cafe,Menu,Price,Address
Rank,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,"
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,"
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,"
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,"
5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,"


In [260]:
gmaps_key = '~~'
gmaps = googlemaps.Client(key=gmaps_key)

In [261]:
# 여러곳에 가맹점이 있기에 특정주소가 없는 행 6개
np.sum(df['Address']=='Multiple') 

6

In [262]:
lat = []
lng = []
for n in tqdm_notebook(df.index): # 0~49
    if df['Address'][n] != 'Multiple':
        target_name = df['Address'][n]+', '+'Chicago'
        gamps_output = gmaps.geocode(target_name) # 구글맵에 검색된 정보를 반환
        
        # 구글맵에 던지면 위도경도 반환
        location_output = gamps_output[0].get('geometry') 
        print(location_output)
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
    else:
        lat.append(np.nan)
        lng.append(np.nan)
# print(f'{lat}, {lng}')

{'bounds': {'northeast': {'lat': 41.8957463, 'lng': -87.6798563}, 'southwest': {'lat': 41.8954846, 'lng': -87.6800603}}, 'location': {'lat': 41.8955577, 'lng': -87.6799673}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': 41.89702008029149, 'lng': -87.67860931970849}, 'southwest': {'lat': 41.8943221197085, 'lng': -87.6813072802915}}}
{'location': {'lat': 41.8846392, 'lng': -87.6475897}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': 41.88591923029149, 'lng': -87.64623426970849}, 'southwest': {'lat': 41.8832212697085, 'lng': -87.6489322302915}}}
{'bounds': {'northeast': {'lat': 41.8905932, 'lng': -87.6305809}, 'southwest': {'lat': 41.8904555, 'lng': -87.6309496}}, 'location': {'lat': 41.8905226, 'lng': -87.6307834}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': 41.8918733302915, 'lng': -87.6294866697085}, 'southwest': {'lat': 41.8891753697085, 'lng': -87.6321846302915}}}
{'location': {'lat': 42.0583217, 'lng': -87.6837479}, 'location_type': '

In [265]:
df['lat'] = lat
df['lng'] = lng
df.head()

,Cafe,Menu,Price,Address,lat,lng
Rank,,,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,",41.895558,-87.679967
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,",41.884639,-87.647590
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,",41.890523,-87.630783
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,",42.058322,-87.683748
5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,",41.886604,-87.648536


In [266]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()]
                    ,zoom_start=11)
folium.Marker([df['lat'].mean(), df['lng'].mean()]
             ,popup='center').add_to(mapping)
mapping

In [301]:
df['lat']>0

Rank
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12    False
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22    False
23     True
24     True
25    False
26     True
27    False
28     True
29     True
30     True
31    False
32     True
33     True
34     True
35     True
36     True
37     True
38     True
39    False
40     True
41     True
42     True
43     True
44     True
45     True
46     True
47     True
48     True
49     True
50     True
Name: lat, dtype: bool

In [311]:
map = folium.Map(location=[df['lat'].mean(), df['lng'].mean()],zoom_start=11)

for i in range(len(df['lat'])):
    if df['lat'].iloc[i]>0:
        folium.Marker([df['lat'].iloc[i], df['lng'].iloc[i]]
                      ,popup=df['Cafe'].iloc[i]).add_to(map)
map


In [317]:
from folium.plugins import MarkerCluster
mc = MarkerCluster()

map = folium.Map(location=[df['lat'].mean(), df['lng'].mean()],zoom_start=11)

for i in df.index:
    if df['Address'][i] != 'Multiple':
        mc.add_child(folium.Marker(location=[df['lat'][i], df['lng'][i]]
                                  ,popup=df['Cafe'][i]))
map.add_child(mc)
map

## 참고 - BeautifulSoup 문법 다루기

In [89]:
# HTML Parser가 기본적으로 제공
page = open('../data/03. test_first.html',"r").read()
soup= BeautifulSoup(page, 'html.parser')
print(soup.prettify()) # html 페이지의 내용을 전체 다 보고 싶으면

<!DOCTYPE html>
<html>
 <head>
  <title>
   Very Simple HTML Code by PinkWink
  </title>
 </head>
 <body>
  <div>
   <p class="inner-text first-item" id="first">
    Happy PinkWink.
    <a href="http://www.pinkwink.kr" id="pw-link">
     PinkWink
    </a>
   </p>
   <p class="inner-text second-item">
    Happy Data Science.
    <a href="https://www.python.org" id="py-link">
     Python
    </a>
   </p>
  </div>
  <p class="outer-text first-item" id="second">
   <b>
    Data Science is funny.
   </b>
  </p>
  <p class="outer-text">
   <b>
    All I need is Love.
   </b>
  </p>
 </body>
</html>


####  9-1. preottify() : html 페이지의 내용을 전체 다 보고 싶으면

In [ ]:
print(soup.prettify()) 


In [15]:
html = list(soup.children)[2]
html

<html>
<head>
<title>Very Simple HTML Code by PinkWink</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                Happy PinkWink.
                <a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>
</p>
<p class="inner-text second-item">
                Happy Data Science.
                <a href="https://www.python.org" id="py-link">Python</a>
</p>
</div>
<p class="outer-text first-item" id="second">
<b>
                Data Science is funny.
            </b>
</p>
<p class="outer-text">
<b>
                All I need is Love.
            </b>
</p>
</body>
</html>

In [29]:
list(html.children)

['\n', <head>
 <title>Very Simple HTML Code by PinkWink</title>
 </head>, '\n', <body>
 <div>
 <p class="inner-text first-item" id="first">
                 Happy PinkWink.
                 <a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>
 </p>
 <p class="inner-text second-item">
                 Happy Data Science.
                 <a href="https://www.python.org" id="py-link">Python</a>
 </p>
 </div>
 <p class="outer-text first-item" id="second">
 <b>
                 Data Science is funny.
             </b>
 </p>
 <p class="outer-text">
 <b>
                 All I need is Love.
             </b>
 </p>
 </body>, '\n']

####  9-2. html의 children을 총 3번 조사하면 body태그가 나온다. 

In [31]:
body = list(html.children)[3]
body

<body>
<div>
<p class="inner-text first-item" id="first">
                Happy PinkWink.
                <a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>
</p>
<p class="inner-text second-item">
                Happy Data Science.
                <a href="https://www.python.org" id="py-link">Python</a>
</p>
</div>
<p class="outer-text first-item" id="second">
<b>
                Data Science is funny.
            </b>
</p>
<p class="outer-text">
<b>
                All I need is Love.
            </b>
</p>
</body>

In [34]:
list(body.children)

['\n', <div>
 <p class="inner-text first-item" id="first">
                 Happy PinkWink.
                 <a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>
 </p>
 <p class="inner-text second-item">
                 Happy Data Science.
                 <a href="https://www.python.org" id="py-link">Python</a>
 </p>
 </div>, '\n', <p class="outer-text first-item" id="second">
 <b>
                 Data Science is funny.
             </b>
 </p>, '\n', <p class="outer-text">
 <b>
                 All I need is Love.
             </b>
 </p>, '\n']

####  9-3. body태그를 한번에 나타낼수도 있다

In [38]:
soup.body

<body>
<div>
<p class="inner-text first-item" id="first">
                Happy PinkWink.
                <a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>
</p>
<p class="inner-text second-item">
                Happy Data Science.
                <a href="https://www.python.org" id="py-link">Python</a>
</p>
</div>
<p class="outer-text first-item" id="second">
<b>
                Data Science is funny.
            </b>
</p>
<p class="outer-text">
<b>
                All I need is Love.
            </b>
</p>
</body>

####  9-4. find(), find_all()
- <font color='blue'>find_all(): 해당태그 전체 찾기 </font>
- find() : 첫번째 @태그 찾기

<font color='blue'>class_='클래스이름'</font>

<font color='blue'>id='id명'</font>

In [36]:
soup.find_all('p')

[<p class="inner-text first-item" id="first">
                 Happy PinkWink.
                 <a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>
 </p>, <p class="inner-text second-item">
                 Happy Data Science.
                 <a href="https://www.python.org" id="py-link">Python</a>
 </p>, <p class="outer-text first-item" id="second">
 <b>
                 Data Science is funny.
             </b>
 </p>, <p class="outer-text">
 <b>
                 All I need is Love.
             </b>
 </p>]

In [48]:
print(soup.find_all('p',class_='outer-text'))
print("*"*40)
print("*"*40)
print(soup.find_all(class_='outer-text'))

[<p class="outer-text first-item" id="second">
<b>
                Data Science is funny.
            </b>
</p>, <p class="outer-text">
<b>
                All I need is Love.
            </b>
</p>]
****************************************
****************************************
[<p class="outer-text first-item" id="second">
<b>
                Data Science is funny.
            </b>
</p>, <p class="outer-text">
<b>
                All I need is Love.
            </b>
</p>]


In [50]:
soup

<!DOCTYPE html>

<html>
<head>
<title>Very Simple HTML Code by PinkWink</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                Happy PinkWink.
                <a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>
</p>
<p class="inner-text second-item">
                Happy Data Science.
                <a href="https://www.python.org" id="py-link">Python</a>
</p>
</div>
<p class="outer-text first-item" id="second">
<b>
                Data Science is funny.
            </b>
</p>
<p class="outer-text">
<b>
                All I need is Love.
            </b>
</p>
</body>
</html>

In [49]:
soup.find_all(id="first")

[<p class="inner-text first-item" id="first">
                 Happy PinkWink.
                 <a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>
 </p>]

#### 9-5. sibling

In [63]:
soup

<!DOCTYPE html>

<html>
<head>
<title>Very Simple HTML Code by PinkWink</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                Happy PinkWink.
                <a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>
</p>
<p class="inner-text second-item">
                Happy Data Science.
                <a href="https://www.python.org" id="py-link">Python</a>
</p>
</div>
<p class="outer-text first-item" id="second">
<b>
                Data Science is funny.
            </b>
</p>
<p class="outer-text">
<b>
                All I need is Love.
            </b>
</p>
</body>
</html>

In [59]:
soup.p

<p class="inner-text first-item" id="first">
                Happy PinkWink.
                <a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>
</p>

In [60]:
soup.p.next_sibling

'\n'

In [62]:
soup.p.next_sibling.next_sibling

<p class="inner-text second-item">
                Happy Data Science.
                <a href="https://www.python.org" id="py-link">Python</a>
</p>

In [64]:
for each_tag in soup.find_all('p'):
    print(each_tag.get_text())


                Happy PinkWink.
                PinkWink


                Happy Data Science.
                Python



                Data Science is funny.
            



                All I need is Love.
            



In [75]:
for each_tag in soup.find_all('b'):
    print(each_tag.get_text())


                Data Science is funny.
            

                All I need is Love.
            


In [77]:
links = soup.find_all('a')
links

[<a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>,
 <a href="https://www.python.org" id="py-link">Python</a>]

In [86]:
links[0]

<a href="http://www.pinkwink.kr" id="pw-link">PinkWink</a>

In [88]:
for each in links:
    href=each['href'] # 접근이 되네 ㄷㄷ;
    text = each.string
    print(f'{text} -> {href}')

PinkWink -> http://www.pinkwink.kr
Python -> https://www.python.org


#### -9-6. 크롬 개발자 도구를 이용해서 원하는 태그 찾기

In [101]:
url = "https://finance.naver.com/marketindex/"
page = urlopen(url)

soup = BeautifulSoup(page, "html.parser")

print(soup.prettify())


<script language="javascript" src="/template/head_js.naver?referer=info.finance.naver.com&amp;menu=marketindex&amp;submenu=market">
</script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20220719195125/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript">
</script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20220719195125/js/jindo.1.5.3.element-text-patch.js" type="text/javascript">
</script>
<div id="container" style="padding-bottom:0px;">
 <div class="market_include">
  <div class="market_data">
   <div class="market1">
    <div class="title">
     <h2 class="h_market1">
      <span>
       환전 고시 환율
      </span>
     </h2>
    </div>
    <!-- data -->
    <div class="data">
     <ul class="data_lst" id="exchangeList">
      <li class="on">
       <a class="head usd" href="/marketindex/exchangeDetail.naver?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
        <h3 class="h_lst">
         <span class="blind">
          미국 U

In [100]:
# 다러 환율 가져오자 
soup.find_all(class_='value')[0].get_text()

'1,309.00'